In [7]:
import numpy as np
import matplotlib.pyplot as plt
import os
from skimage import transform
import matplotlib.pyplot as plt
import matplotlib
from IPython.core.pylabtools import figsize # import figsize
import numpy as np
from tifffile import imsave, imread

In [8]:
plt.rcParams['figure.figsize'] = (16, 9)

In [9]:
file_path_image = './WTeSe_450x450/'
file_path_label1 = './Label_Se_450x450/'
file_path_label2 = './Label_vacancy_450x450/'
files_image = os.listdir(file_path_image)
files_label1 = os.listdir(file_path_label1)
files_label2 = os.listdir(file_path_label2)

In [43]:
for i in range(len(files_image)):
    if files_label1[i][3:] != files_image[i] or files_label2[i][8:] != files_image[i]:
        print("WRONG")

In [21]:
temp_file = []
for file1 in files_label2:
    if file1[8:] in files_image:
        temp_file.append(file1[8:])

In [22]:
for file2 in files_image:
    if file2 not in temp_file:
        files_image.remove(file2)
        files_label1.remove("Se_" + file2)
        

In [28]:
temp_file = []
for file1 in files_label1:
    if file1[3:] in files_image:
        temp_file.append(file1[3:])

In [30]:
for file2 in files_image:
    if file2 not in temp_file:
        files_image.remove(file2)
        files_label2.remove("vacancy_" + file2)
        

In [40]:
for file3 in files_label2:
    if file3[8:] not in files_image:
        files_label2.remove(file3)

In [53]:
bkg_stack = imread('Bkg_stack.tif')
np.shape(bkg_stack)

(9, 1024, 1024)

In [44]:
def put_image_in_stack(file_image_list,file_label_list1,file_label_list2
                       ,file_image_path,file_label_path1,file_label_path2):
    '''
    Input:
    file_path: image file dictionary 
    real_image_list: list of real image file names
    defect_list: list of defect names
    Return:
    image_stack: np.array shape = (num_image, x,y,num_defects+1)
    '''
    img = plt.imread(file_image_path+file_image_list[0])
    file_num = len(file_image_list)
    x,y = np.shape(img)
    image_stacks = np.zeros((file_num,x,y,2))
    for i in range(file_num):
        image_stacks[i,:,:,0] = plt.imread(file_image_path+file_image_list[i])
        image_stacks[i,:,:,1] = plt.imread(file_label_path1+file_label_list1[i])+plt.imread(file_label_path2+file_label_list2[i])
        if i>500 and i%100 == 0:
            print("put {} in stack".format(i))
    return image_stacks

In [45]:
def _get_normal_distribution(param):
    '''
    param: (mean,std)
    return: normal_distribution
    '''
    num = np.random.normal(param[0],param[1])
    pos = np.random.rand()
    if pos>0.5:
        return num
    else:
        return -num
def postProcess1(image_stacks,ifShear = True,ifConstrain = True,ifJittering = True,
                shear_rate = (0,0), constrain_rate = (1,0)):
    '''
    Input:
    image_stacks: np.arrays shape (num_images,x,y,num_defects+1)
    ifShear,ifConstrain,ifJittering: boolean, if add these on your images
    shear_rate, constrain_rate,jittering_rate: tuple, (mean,std) for gaussian distrbution
    
    Return :
    image_stacks: same size with the input, after process
    '''
    image_stacks_copy = image_stacks.copy()
    num_image, x, y, num_defects = np.shape(image_stacks)
    
    shear = 0
    constrain = 1
    jittering_list = np.array([-2,-1,-1,0,1,1,2])

        
    for i in range(num_image):
        temp_stack = image_stacks_copy[i,:,:,:]
        
        if ifShear:
            shear = _get_normal_distribution(shear_rate)
        
        if ifConstrain:
            constrain = _get_normal_distribution(constrain_rate)+1
        afine_tf = transform.AffineTransform(scale = (1,constrain),shear = shear)
        modified = transform.warp(temp_stack, inverse_map=afine_tf)
       
        image_stacks_copy[i,:,:,:] = modified
    return image_stacks_copy

In [46]:
def postProcess2(image_stack_original, target_image,bkg_stack,gaussian_rate = (0,0)):
    '''
    Input:
    image_stacks: np.arrays shape (num_images,x,y,num_defects+1)
    target_images: np.arrays shape(x,y) real STEM image, for b/c, cropped size
    Return:
    image_stacks:np.arrays shape(num_images,target_x,target_y,numnum_defects+1), 8bits
    '''
    image_stacks = image_stack_original.copy()
    num_images,x,y,num_defects = np.shape(image_stacks)
    
    for i in range(num_images):
        degree = np.random.randint(-5,5)
        for j in range(num_defects):
            image_stacks[i,:,:,j] = transform.rotate(image_stacks[i,:,:,j],degree)
        if i%100 == 0:
            print('finish {} images'.format(i))
            
    
    target_x,targer_y = np.shape(target_image)
    #target_mean, target_std = np.mean(target_image), np.std(target_image)
    cropped_x = abs(x-target_x)//2
    cropped_y = abs(y-targer_y)//2
    image_stacks = image_stacks[:,cropped_x:-cropped_x,cropped_y:-cropped_y,:]
    #print('target_mean', target_mean)
    #print('target_std',target_std)
    x1,y1 = np.shape(image_stacks[0,:,:,0])
    for i in range(num_images):
        image_stacks[i,:,:,0] += np.random.normal(gaussian_rate[0],gaussian_rate[1],(x1,y1))
        r,x,y = np.random.randint(0,9),np.random.randint(0,700),np.random.randint(0,700)
        image_stacks[i,:,:,0] += bkg_stack[r,x:x+x1,y:y+y1]
        
        #temp_std = np.std(temp_image)
        #temp_image *= target_std/temp_std
        #temp_mean = np.mean(temp_image)
        #temp_image += target_mean-temp_mean
        #temp_mean, temp_std = np.mean(temp_image),np.std(temp_image)
        #print('after mean=  ', temp_mean)
        #print('after std = ',temp_std)
    sel = image_stacks<0
    image_stacks[sel] = 0
    image_stacks = (image_stacks-np.min(image_stacks)) /(np.ptp(image_stacks))*255
    
    return image_stacks
            
    

In [47]:
target_path = './target_image/'

In [48]:
target_image = plt.imread(target_path+'2254 HAADF 4.6 Mx HAADF.tif')[100:356,100:356]

In [49]:
image_stack_original = put_image_in_stack(files_image,files_label1,files_label2,
                                          file_path_image,file_path_label1,file_path_label2)

put 600 in stack
put 700 in stack
put 800 in stack
put 900 in stack
put 1000 in stack
put 1100 in stack
put 1200 in stack
put 1300 in stack
put 1400 in stack
put 1500 in stack
put 1600 in stack
put 1700 in stack
put 1800 in stack
put 1900 in stack
put 2000 in stack
put 2100 in stack
put 2200 in stack
put 2300 in stack
put 2400 in stack
put 2500 in stack
put 2600 in stack
put 2700 in stack
put 2800 in stack
put 2900 in stack


In [50]:
np.shape(image_stack_original)

(2963, 450, 455, 2)

In [51]:
image_stacks_1 = postProcess1(image_stack_original,shear_rate=(0.05,0.025),
                             constrain_rate=(0.05,0.025))

In [54]:
image_stacks_2  = postProcess2(image_stacks_1,target_image,bkg_stack,gaussian_rate=(0,20))

finish 0 images
finish 100 images
finish 200 images
finish 300 images
finish 400 images
finish 500 images
finish 600 images
finish 700 images
finish 800 images
finish 900 images
finish 1000 images
finish 1100 images
finish 1200 images
finish 1300 images
finish 1400 images
finish 1500 images
finish 1600 images
finish 1700 images
finish 1800 images
finish 1900 images
finish 2000 images
finish 2100 images
finish 2200 images
finish 2300 images
finish 2400 images
finish 2500 images
finish 2600 images
finish 2700 images
finish 2800 images
finish 2900 images


In [55]:
defect_list = ['Se+V']

In [56]:
import tifffile
def saveImageStack(image_stacks,defect_list,target_image,fold_name):
    
    os.makedirs(fold_name)
    num_images,x,y,num_defects = np.shape(image_stacks)
  
    for i in range(num_images):
        temp_path = fold_name+'image{}/'.format(i)
        os.makedirs(temp_path)
        tifffile.imsave(temp_path+'training_image{}'.format(i)+'.tiff',image_stacks[i,:,:,0].astype('uint8'))
        for j in range(len(defect_list)):
            tifffile.imsave(temp_path+'image{}_label'.format(i)+defect_list[j]+'.tiff',
                            image_stacks[i,:,:,j+1].astype('uint8'))
        

In [57]:
import tifffile
def saveImageStack1(image_stacks,defect_list,fold_name):
    
    os.makedirs(fold_name)
    for defect in defect_list:
        os.makedirs(fold_name+defect)
    num_images,x,y,num_defects = np.shape(image_stacks)
  
    for i in range(num_images):
        
        
        tifffile.imsave(fold_name+'/'+'training_image{}'.format(i)+'.tiff',image_stacks[i,:,:,0].astype('uint8'))
        for j in range(len(defect_list)):
            tifffile.imsave(fold_name+defect_list[j]+'/'+'image{}_label'.format(i)+defect_list[j]+'.tiff',
                            image_stacks[i,:,:,j+1].astype('uint8'))
        

In [58]:
saveImageStack1(image_stacks_2,defect_list,'./final_output_bkg_Se+V')

In [59]:
np.save('./image_256x256_bkg_Se+V_image.npy',image_stacks_2[:,:,:,0])

In [60]:
np.save('./image_256x256_Se+V_bkg_label.npy',image_stacks_2[:,:,:,1])

In [53]:
for i in range(len(files_label)):
    if files_label[i][3:] == files_image[i]:
        print(files_label[i][3:])

WSeTe_incostem_28_16_1_0.tif
WSeTe_incostem_28_16_1_1.tif
WSeTe_incostem_28_16_1_10.tif
WSeTe_incostem_28_16_1_100.tif
WSeTe_incostem_28_16_1_1000.tif
WSeTe_incostem_28_16_1_1001.tif
WSeTe_incostem_28_16_1_1002.tif
WSeTe_incostem_28_16_1_1003.tif
WSeTe_incostem_28_16_1_1004.tif
WSeTe_incostem_28_16_1_1005.tif
WSeTe_incostem_28_16_1_1006.tif
WSeTe_incostem_28_16_1_1007.tif
WSeTe_incostem_28_16_1_1008.tif
WSeTe_incostem_28_16_1_1009.tif
WSeTe_incostem_28_16_1_101.tif
WSeTe_incostem_28_16_1_1010.tif
WSeTe_incostem_28_16_1_1011.tif
WSeTe_incostem_28_16_1_1012.tif
WSeTe_incostem_28_16_1_1013.tif
WSeTe_incostem_28_16_1_1014.tif
WSeTe_incostem_28_16_1_1015.tif
WSeTe_incostem_28_16_1_1016.tif
WSeTe_incostem_28_16_1_1017.tif
WSeTe_incostem_28_16_1_1018.tif
WSeTe_incostem_28_16_1_1019.tif
WSeTe_incostem_28_16_1_102.tif
WSeTe_incostem_28_16_1_1020.tif
WSeTe_incostem_28_16_1_1021.tif
WSeTe_incostem_28_16_1_1022.tif
WSeTe_incostem_28_16_1_1023.tif
WSeTe_incostem_28_16_1_1024.tif
WSeTe_incostem_28_1

WSeTe_incostem_28_16_1_2387.tif
WSeTe_incostem_28_16_1_2388.tif
WSeTe_incostem_28_16_1_2389.tif
WSeTe_incostem_28_16_1_239.tif
WSeTe_incostem_28_16_1_2390.tif
WSeTe_incostem_28_16_1_2391.tif
WSeTe_incostem_28_16_1_2392.tif
WSeTe_incostem_28_16_1_2393.tif
WSeTe_incostem_28_16_1_2394.tif
WSeTe_incostem_28_16_1_2395.tif
WSeTe_incostem_28_16_1_2396.tif
WSeTe_incostem_28_16_1_2397.tif
WSeTe_incostem_28_16_1_2398.tif
WSeTe_incostem_28_16_1_2399.tif
WSeTe_incostem_28_16_1_24.tif
WSeTe_incostem_28_16_1_240.tif
WSeTe_incostem_28_16_1_2400.tif
WSeTe_incostem_28_16_1_2401.tif
WSeTe_incostem_28_16_1_2402.tif
WSeTe_incostem_28_16_1_2403.tif
WSeTe_incostem_28_16_1_2404.tif
WSeTe_incostem_28_16_1_2405.tif
WSeTe_incostem_28_16_1_2406.tif
WSeTe_incostem_28_16_1_2407.tif
WSeTe_incostem_28_16_1_2408.tif
WSeTe_incostem_28_16_1_2409.tif
WSeTe_incostem_28_16_1_241.tif
WSeTe_incostem_28_16_1_2410.tif
WSeTe_incostem_28_16_1_2411.tif
WSeTe_incostem_28_16_1_2412.tif
WSeTe_incostem_28_16_1_2413.tif
WSeTe_incoste

WSeTe_incostem_28_16_1_63.tif
WSeTe_incostem_28_16_1_630.tif
WSeTe_incostem_28_16_1_631.tif
WSeTe_incostem_28_16_1_632.tif
WSeTe_incostem_28_16_1_633.tif
WSeTe_incostem_28_16_1_634.tif
WSeTe_incostem_28_16_1_635.tif
WSeTe_incostem_28_16_1_636.tif
WSeTe_incostem_28_16_1_637.tif
WSeTe_incostem_28_16_1_638.tif
WSeTe_incostem_28_16_1_639.tif
WSeTe_incostem_28_16_1_64.tif
WSeTe_incostem_28_16_1_640.tif
WSeTe_incostem_28_16_1_641.tif
WSeTe_incostem_28_16_1_642.tif
WSeTe_incostem_28_16_1_643.tif
WSeTe_incostem_28_16_1_644.tif
WSeTe_incostem_28_16_1_645.tif
WSeTe_incostem_28_16_1_646.tif
WSeTe_incostem_28_16_1_647.tif
WSeTe_incostem_28_16_1_648.tif
WSeTe_incostem_28_16_1_649.tif
WSeTe_incostem_28_16_1_65.tif
WSeTe_incostem_28_16_1_650.tif
WSeTe_incostem_28_16_1_651.tif
WSeTe_incostem_28_16_1_652.tif
WSeTe_incostem_28_16_1_653.tif
WSeTe_incostem_28_16_1_654.tif
WSeTe_incostem_28_16_1_655.tif
WSeTe_incostem_28_16_1_656.tif
WSeTe_incostem_28_16_1_657.tif
WSeTe_incostem_28_16_1_658.tif
WSeTe_incos